In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from pandas.io.json import json_normalize
import json
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim


print('Imported Required Libraries')

Imported Required Libraries


In [2]:
site = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(site, 'html.parser')

In [3]:
postalCodeList = []
boroughList = []
neighborhoodList = []

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [4]:
toronto_neighorhood = [('PostalCode', postalCodeList),
                      ('Borough', boroughList),
                      ('Neighborhood', neighborhoodList)]
toronto = pd.DataFrame.from_dict(dict(toronto_neighorhood))
toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [5]:
toronto_na = toronto[toronto.Borough != 'Not assigned\n'].reset_index(drop=True)
toronto_na['PostalCode'] = toronto_na['PostalCode'].astype(str).str[:-1].astype(str)
toronto_na.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York\n,Parkwoods
1,M4A,North York\n,Victoria Village
2,M5A,Downtown Toronto\n,"Regent Park, Harbourfront"
3,M6A,North York\n,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government"


In [6]:
toronto_na.shape

(103, 3)

In [7]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates downloaded!')
coors = pd.read_csv('toronto_coordinates.csv')

Coordinates downloaded!


In [8]:
print(coors.shape)
coors.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
toronto_df_temp = toronto_na.set_index('PostalCode')
coors_temp = coors.set_index('Postal Code')
toronto_df_coors = pd.concat([toronto_df_temp, coors_temp], axis=1, join='inner')

In [12]:
toronto_df_coors.index.name = 'PostalCode'
toronto_df_coors.reset_index(inplace=True)

In [13]:
print(toronto_df_coors.shape)
toronto_df_coors.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York\n,Parkwoods,43.753259,-79.329656
1,M4A,North York\n,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto\n,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York\n,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
